In [1]:
import os
import zipfile
import pandas as pd
import tensorflow as tf
import numpy as np
from PIL import Image

In [13]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [2]:
dataset_zip = 'Acne detection.v1i.multiclass.zip'
extract_dir = 'data2/'

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [3]:
train_csv_path = os.path.join(extract_dir, 'train', '_classes.csv')
valid_csv_path = os.path.join(extract_dir, 'valid', '_classes.csv')
test_csv_path = os.path.join(extract_dir, 'test', '_classes.csv')

In [14]:
img_height = 512
img_width = 512
batch_size = 8 

In [15]:
def load_image(img_path, label):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    image = image / 255.0  # Normalize to [0,1] range
    return image, label

# Function to create a TensorFlow dataset from CSV
def create_dataset(csv_path, base_dir, batch_size=32):
    data = pd.read_csv(csv_path)
    filepaths = data['filename'].apply(lambda x: os.path.join(base_dir, x)).values
    labels = data.iloc[:, 1:].values.astype('float32')
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(filepaths)).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

In [16]:
train_dataset = create_dataset(train_csv_path, os.path.join(extract_dir, 'train'), batch_size)
valid_dataset = create_dataset(valid_csv_path, os.path.join(extract_dir, 'valid'), batch_size)
test_dataset = create_dataset(test_csv_path, os.path.join(extract_dir, 'test'), batch_size)

In [17]:
for image_batch, label_batch in train_dataset.take(1):
    print(image_batch.shape, label_batch.shape)

ResourceExhaustedError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[1,512,512,3] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node resize/ResizeBilinear}}]] [Op:IteratorGetNext] name: 